# FireNet to detect fire

In [6]:
from imageai.Detection.Custom import CustomObjectDetection, CustomVideoObjectDetection
import os

execution_path = os.getcwd()


def train_detection_model():
    from imageai.Detection.Custom import DetectionModelTrainer

    trainer = DetectionModelTrainer()
    trainer.setModelTypeAsYOLOv3()
    trainer.setDataDirectory(data_directory="fire-dataset")
    trainer.setTrainConfig(object_names_array=["fire"], batch_size=8, num_experiments=100,
                           train_from_pretrained_model="pretrained-yolov3.h5")
    # download 'pretrained-yolov3.h5' from the link below
    # https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5
    trainer.trainModel()


def detect_from_image(imgName):
    detector = CustomObjectDetection()
    detector.setModelTypeAsYOLOv3()
    detector.setModelPath(detection_model_path=os.path.join(execution_path, "detection_model-ex-33--loss-4.97.h5"))
    detector.setJsonPath(configuration_json=os.path.join(execution_path, "detection_config.json"))
    detector.loadModel()

    detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path, imgName),
                                                 output_image_path=os.path.join(execution_path, "1-detected.jpg"),
                                                 minimum_percentage_probability=40)

    detected = False
    for detection in detections:
        detected = True
        print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])
        
    #detector.summary()
    
    if(not detected):
        print('no fire detected')
        return 0;
    else:
        return detection["percentage_probability"];

def detect_from_video():
    detector = CustomVideoObjectDetection()
    detector.setModelTypeAsYOLOv3()
    detector.setModelPath(detection_model_path=os.path.join(execution_path, "detection_model-ex-33--loss-4.97.h5"))
    detector.setJsonPath(configuration_json=os.path.join(execution_path, "detection_config.json"))
    detector.loadModel()

    detected_video_path = detector.detectObjectsFromVideo(input_file_path=os.path.join(execution_path, "video2.mp4"), frames_per_second=30, output_file_path=os.path.join(execution_path, "video2-detected"), minimum_percentage_probability=40, log_progress=True )




# Get Image from S3

###### Get the image from S3 bucket to local storage for verifying the fire by using FireNet
###### if probabality > threshold, then it send SMS with the URL of the image

In [3]:
import requests

def verifyIfFire(URL):
    with open('latest.png', 'wb') as handle:
            response = requests.get(URL, stream=True)

            if not response.ok:
                print(response)

            for block in response.iter_content(1024):
                if not block:
                    break

                handle.write(block)
                
    detection = detect_from_image("latest.png")

    if(detection > 30):
        send_sms("Fire More than the threshold detected ", URL)
        
#verifyIfFire("https://fire-detection-ads.s3.amazonaws.com/124.png")

# MQTT Client Subscriber

#### Always listning to an event from pi. If Pi-Model says fire detected, then it verify it first with FireNet and then send the sms

In [ ]:
import paho.mqtt.client as mqtt
 
MQTT_SERVER = "192.168.1.31"
MQTT_PATH = "test_channel"
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
 
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(MQTT_PATH)
 
# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload))
    if("FIRE DETECTED" in str(msg.payload)):
        #verify fire
        URL = msg.payload[14:].decode("utf-8") 
        print(URL);
        verifyIfFire(URL) #verify functions -> checks fire probablity -> if higher than threshold -> sends sms 
        
    # more callbacks, etc
 
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
 
client.connect(MQTT_SERVER, 1883, 60)
 
# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()


Connected with result code 0
test_channel b'FIRE DETECTED https://fire-detection-ads.s3.amazonaws.com/268.png'
https://fire-detection-ads.s3.amazonaws.com/268.png
no fire detected
test_channel b'FIRE DETECTED https://fire-detection-ads.s3.amazonaws.com/269.png'
https://fire-detection-ads.s3.amazonaws.com/269.png
no fire detected
test_channel b'FIRE DETECTED https://fire-detection-ads.s3.amazonaws.com/270.png'
https://fire-detection-ads.s3.amazonaws.com/270.png
no fire detected
test_channel b'FIRE DETECTED https://fire-detection-ads.s3.amazonaws.com/271.png'
https://fire-detection-ads.s3.amazonaws.com/271.png
no fire detected
test_channel b'FIRE DETECTED https://fire-detection-ads.s3.amazonaws.com/272.png'
https://fire-detection-ads.s3.amazonaws.com/272.png
no fire detected
Connected with result code 0
test_channel b'FIRE DETECTED https://fire-detection-ads.s3.amazonaws.com/300.png'
https://fire-detection-ads.s3.amazonaws.com/300.png
no fire detected
test_channel b'FIRE DETECTED https:/

# Send SMS

In [4]:
import boto3
from botocore.exceptions import NoCredentialsError
from boto.s3.connection import S3Connection

def send_sms(message, filename):

        print('Sending sms')
        # Create an SNS client
        client = boto3.client(
                "sns",
                aws_access_key_id="",
                region_name="us-east-1"
        )

# Create the topic if it doesn't exist (this is idempotent)
# Send your sms message.
        response = client.publish(
            PhoneNumber="+17063088069",
            Message="Alert 🚨 : "+message+" View the captured image by clicking on this link : "+filename
  
        )
        if(response):
               print("Message sent")
        else:
               print("Message not sent")
                
